In [5]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [11]:
import asyncio 
from google import genai
from google.genai import types
import nest_asyncio
nest_asyncio.apply()

client = genai.Client()
model = "gemini-live-2.5-flash-preview"


def open_shop():
    return "opened"

def close_shop():
    return "closed"




shop_open = types.FunctionDeclaration(name="open_shop",behavior="NON_BLOCKING")
shop_close = types.FunctionDeclaration(name="close_shop")


tools = [types.Tool(function_declarations=[shop_open, shop_close])]


#tools = [{"function_declarations":[shop_open,shop_close]}]
config = types.LiveConnectConfig(
     response_modalities = ["TEXT"],
     tools=tools
)


async def main():
    async with client.aio.live.connect(model=model, config = config) as session:
        prompt = "open the shop and tell me what happened"
        await session.send_client_content(turns ={"parts":[{"text":prompt}]})

        async for chunk in session.receive():
            #print(chunk)
            if chunk.server_content:
                if chunk.text is not None:
                    print(chunk.text,end="")
            if chunk.tool_call:
                function_responses = []
                for fc in chunk.tool_call.function_calls:
                    #print(fc)
                    result = open_shop() if fc.name == "open_shop" else close_shop()
                    function_response = types.FunctionResponse(
                        id = fc.id,
                        name = fc.name,
                        response = {"result":result,"scheduling":"WHEN_IDLE"}
                    )
                    function_responses.append(function_response)
                await session.send_tool_response(function_responses=function_responses)
                #await session.send_tool_response(function_responses)

await main()

id='function-call-11038371609359629807' args={} name='open_shop'
The shop is now open.